In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-09-23 13:50:42--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-09-23 13:50:42 (132 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
#import helper functions we're going to use in this notebook

from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [3]:
import os

for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/Tez Dosyaları/part 1"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '/content/drive/MyDrive/Tez Dosyaları/part 1'.
There are 2 directories and 0 images in '/content/drive/MyDrive/Tez Dosyaları/part 1/Test'.
There are 0 directories and 17 images in '/content/drive/MyDrive/Tez Dosyaları/part 1/Test/Benign'.
There are 0 directories and 56 images in '/content/drive/MyDrive/Tez Dosyaları/part 1/Test/Malignant'.
There are 2 directories and 0 images in '/content/drive/MyDrive/Tez Dosyaları/part 1/Train'.
There are 0 directories and 222 images in '/content/drive/MyDrive/Tez Dosyaları/part 1/Train/Malignant'.
There are 0 directories and 71 images in '/content/drive/MyDrive/Tez Dosyaları/part 1/Train/Benign'.


In [4]:
train_dir = "/content/drive/MyDrive/Tez Dosyaları/part 1/Train"
test_dir = "/content/drive/MyDrive/Tez Dosyaları/part 1/Test"

In [5]:
import tensorflow as tf
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_data= tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                          label_mode="binary",
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                          label_mode="binary",
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE)

Found 293 files belonging to 2 classes.
Found 73 files belonging to 2 classes.


In [6]:
train_data.class_names

['Benign', 'Malignant']

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import random
import math
import csv
import cv2
import os

In [8]:
image_width = 224
image_height = 224
classes = ['Benign', 'Malignant']
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Resimleri yüklemek için flow_from_directory kullanın
train_generator = datagen.flow_from_directory(
    directory="/content/drive/MyDrive/Tez Dosyaları/part 1/Train",
    target_size=(image_width, image_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Her sınıf için resimleri ayrı ayrı arttırın
for cls in classes:
    # Sınıfın resimlerini yüklemek için flow_from_directory kullanın
    cls_generator = datagen.flow_from_directory(
        directory="/content/drive/MyDrive/Tez Dosyaları/part 1/Train",
        target_size=(image_width, image_height),
        batch_size=32,
        class_mode='categorical',
        subset='training',
        classes=[cls]
    )

    # Resimleri arttırın ve kaydedin
    i = 0
    for batch in cls_generator:
        images, labels = batch
        for image in images:
            # Resmi kaydetmek için imwrite kullanın
            cv2.imwrite(os.path.join("/content/drive/MyDrive/Tez Dosyaları/part 1/Train", cls, f"augmented_{i}.jpg"), image * 255)
            i += 1
            if i >= 3000:  # Maksimum resim sayısına ulaşıldığında döngüyü durdur
                break
        else:
            continue
        break

Found 293 images belonging to 2 classes.
Found 71 images belonging to 1 classes.
Found 222 images belonging to 1 classes.


In [9]:
# Load the augmented images for the malignant class
augmented_images = os.listdir("/content/drive/MyDrive/Tez Dosyaları/part 1/Train/Malignant")

# Count the number of augmented images
num_malignant_images = len(augmented_images)

# Print the number of malignant images
print("Number of malignant images:", num_malignant_images)

Number of malignant images: 3222


In [10]:
augmented_images3 = os.listdir("/content/drive/MyDrive/Tez Dosyaları/part 1/Train/Benign")

# Count the number of augmented images
num_benign_images = len(augmented_images3)

# Print the number of malignant images
print("Number of benign images:", num_benign_images)

Number of benign images: 3071


In [11]:
import tensorflow as tf
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_data= tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                          label_mode="binary",
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                          label_mode="binary",
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE)

Found 6293 files belonging to 2 classes.
Found 73 files belonging to 2 classes.


In [22]:
# 1.  create base model with tf.keras.applications
base_model = tf.keras.applications.DenseNet121(include_top=False)

# 2. Freeze the base model
base_model.trainable = False

# 3. create inputs into our model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

# 4. if using ResNet50V2 you will need to normalize inputs
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5. pass the inputs to the base_model
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

# 7. create the output activation layer
outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model_0 = tf.keras.Model(inputs, outputs)

# 8. compile the model
model_0.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# 9. fit the model
history_0 = model_0.fit(train_data,
                        epochs=40,
                        steps_per_epoch=len(train_data),
                        validation_data=None,  # validation'ı kapat
                        callbacks=[create_tensorboard_callback(dir_name="transfer_learning",
                                                              experiment_name="10_percent_feature_extraction")])

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Shape after passing inputs through base model: (None, 7, 7, 1024)
Shape after GlobalAveragePooling2D: (None, 1024)
Saving TensorBoard log files to: transfer_learning/10_percent_feature_extraction/20240923-142318
Epoch 1/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 49s 127ms/step - accuracy: 0.5484 - loss: 0.7357
Epoch 2/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 185us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/40


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - accuracy: 0.6392 - loss: 0.6331
Epoch 4/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 60us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - accuracy: 0.6790 - loss: 0.5942
Epoch 6/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 63us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - accuracy: 0.6946 - loss: 0.5744
Epoch 8/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 63us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - accuracy: 0.7062 - loss: 0.5627
Epoch 10/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 61us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - accuracy: 0.7111 - loss: 0.5596
Epoch 12/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 61us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 13/40
197/197 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - accuracy: 0.7175 - loss: 0.5470
Epoc

In [23]:
loss, accuracy = model_0.evaluate(test_data)
print(f"Loss: {loss:.2f}")
print(f"Accuracy: {accuracy:.2f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.7009 - loss: 0.7327
Loss: 0.73
Accuracy: 0.70


In [24]:
# Get the predicted probabilities for the test set
y_pred_probs = model_0.predict(test_data)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_probs > 0.5).astype(int)

# Get the true labels for the test set
y_true = np.concatenate([y for x, y in test_data], axis=0)

# Calculate precision
from sklearn.metrics import precision_score
precision = precision_score(y_true, y_pred)
print("Precision:", precision)

2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step
Precision: 0.75


In [25]:
from sklearn.metrics import recall_score

# Calculate recall
recall = recall_score(y_true, y_pred)
print("Recall:", recall)

Recall: 0.9107142857142857


In [26]:

from sklearn.metrics import f1_score
# Calculate F1-score
f1 = f1_score(y_true, y_pred)
print("F1-score:", f1)


F1-score: 0.8225806451612904
